Lectura de xml

In [2]:
import pandas as pd
import xml.etree.ElementTree as etree



#tree = etree.parse("tg-fal17.xml")
tree = etree.parse("agh-ggis-spr17.xml")
root = tree.getroot()

columns_problem = ["Name", "NrDays", "SlotsPerDay", "NrWeeks"]
columns_optimization = ["Time", "Room", "Distribution", "Student"]
columns_rooms = ["RoomId", "Capacity", "PreviousRoomIdList", "TravelTimeValueList", "UnavailableDaysList", "UnavailableStartList", "UnavailableLengthList", "UnavailableWeeksList"]
columns_courses = ["CourseId", "ConfigIdList"]
columns_configs = ["ConfigId", "SubpartIdList"]
columns_subparts = ["SubpartId", "ClassIdList"]
columns_classes = ["ClassId", "ClassLimit", "ClassParent"]
columns_distributions = ["DistributionId", "DistributionType", "DistributionRequired", "DistributionPenalty", "ClassesList"]
columns_students = ["StudentId", "CoursesList"]
columns_relation_class_room = ["ClassId", "RoomId", "RoomPenalty"]
columns_relation_class_time = ["ClassId", "TimeDays", "TimeStart", "TimeLength", "TimeWeeks", "TimePenalty"]

df_problem = pd.DataFrame(columns = columns_problem)
df_optimization = pd.DataFrame(columns = columns_optimization)
df_rooms = pd.DataFrame(columns = columns_rooms)
df_courses = pd.DataFrame(columns = columns_courses)
df_configs = pd.DataFrame(columns = columns_configs)
df_subparts = pd.DataFrame(columns = columns_subparts)
df_classes = pd.DataFrame(columns = columns_classes)
df_distributions = pd.DataFrame(columns = columns_distributions)
df_students = pd.DataFrame(columns = columns_students)
df_relation_class_room = pd.DataFrame(columns = columns_relation_class_room)
df_relation_class_time = pd.DataFrame(columns = columns_relation_class_time)




df_problem = df_problem.append({"Name": root.attrib.get("name"), "NrDays": root.attrib.get("nrDays"), "SlotsPerDay": root.attrib.get("slotsPerDay"), "NrWeeks": root.attrib.get("nrWeeks")}, ignore_index = True)

for node in root:
    if node.tag == "optimization":
        df_optimization = df_optimization.append({"Time": node.attrib.get("time"), "Room": node.attrib.get("room"), "Distribution": node.attrib.get("distribution"), "Student": node.attrib.get("student")}, ignore_index = True)
    
    elif node.tag == "rooms":
        for room in node:
            roomid = room.attrib.get("id")
            cap = room.attrib.get("capacity")
            prevroomlist = []
            valuelist = []
            dayslist = []
            startlist = []
            lengthlist = []
            weekslist = []
            if len(list(child.tag for child in room.iter() if child is not room)) > 0:
                for child in room:
                    if child.tag == "travel":
                        prevroomid = child.attrib.get("room")
                        value = child.attrib.get("value")
                        prevroomlist.append(prevroomid)
                        valuelist.append(value)
                    elif child.tag == "unavailable":
                        days = child.attrib.get("days")
                        start = child.attrib.get("start")
                        length = child.attrib.get("length")
                        weeks = child.attrib.get("weeks")
                        dayslist.append(days)
                        startlist.append(start)
                        lengthlist.append(length)
                        weekslist.append(weeks)
            if len(prevroomlist) == 0:
                prevroomlist = None
            if len(valuelist) == 0:
                valuelist = None
            if len(dayslist) == 0:
                dayslist = None
            if len(startlist) == 0:
                startlist = None
            if len(lengthlist) == 0:
                lengthlist = None
            if len(weekslist) == 0:
                weekslist = None
            df_rooms = df_rooms.append(pd.Series([roomid, cap, prevroomlist, valuelist, dayslist, startlist, lengthlist, weekslist], index = columns_rooms), ignore_index = True)
            
    elif node.tag == "courses":
        for course in node:
            #print(course.tag)
            courseid = course.attrib.get("id")
            configlist = []
            for config in course:
                #print(config.tag)
                configid = config.attrib.get("id")
                configlist.append(configid)
                subpartlist = []
                for subpart in config:
                    #print(subpart.tag)
                    subpartid = subpart.attrib.get("id")
                    subpartlist.append(subpartid)
                    claselist = []
                    for clase in subpart:
                        #print(clase.tag)
                        claseid = clase.attrib.get("id")
                        claselist.append(claseid)
                        claselimit = clase.attrib.get("limit")
                        claseparent = clase.attrib.get("parent") if clase.attrib.get("parent") is not None else None
                        df_classes = df_classes.append(pd.Series([claseid, claselimit, claseparent], index = columns_classes), ignore_index = True)
                        if len(list(child.tag for child in clase.iter() if child is not clase)) > 0:
                            for child in clase:
                                #print(child.tag)
                                if child.tag == "room":
                                    roomid = child.attrib.get("id")
                                    roompenalty = child.attrib.get("penalty")
                                    df_relation_class_room = df_relation_class_room.append(pd.Series([claseid, roomid, roompenalty], index = columns_relation_class_room), ignore_index = True)
                                elif child.tag == "time":
                                    timedays = child.attrib.get("days")
                                    timestart = child.attrib.get("start")
                                    timelength = child.attrib.get("length")
                                    timeweeks = child.attrib.get("weeks")
                                    timepenalty = child.attrib.get("penalty")
                                    df_relation_class_time = df_relation_class_time.append(pd.Series([claseid, timedays, timestart, timelength, timeweeks, timepenalty], index = columns_relation_class_time), ignore_index = True)
                    df_subparts = df_subparts.append(pd.Series([subpartid, claselist], index = columns_subparts), ignore_index = True)
                df_configs = df_configs.append(pd.Series([configid, subpartlist], index = columns_configs), ignore_index = True)
            df_courses = df_courses.append(pd.Series([courseid, configlist], index = columns_courses), ignore_index = True)
                                    
    elif node.tag == "distributions":
        contador = 1
        for distribution in node:
            #print(distribution.tag)
            distribid = contador
            distribtype = distribution.attrib.get("type")
            distribrequired = distribution.attrib.get("required") if distribution.attrib.get("required") is not None else None
            distribpenalty = distribution.attrib.get("penalty") if distribution.attrib.get("penalty") is not None else None
            listaclases = []
            for clase in distribution:
                #print(clase.tag)
                listaclases.append(clase.attrib.get("id"))
            df_distributions = df_distributions.append(pd.Series([distribid, distribtype, distribrequired, distribpenalty, listaclases], index = columns_distributions), ignore_index = True)
            contador += 1
             
    elif node.tag == "students":
        for student in node:
            studentid = student.attrib.get("id")
            listacourses = []
            for course in student:
                listacourses.append(course.attrib.get("id"))
            df_students = df_students.append(pd.Series([studentid, listacourses], index = columns_students), ignore_index = True)
            
            


print("################################################################################################################")
print("DataFrame Problem")
display(df_problem)
print("################################################################################################################")
print("DataFrame Optimization")
display(df_optimization)
print("################################################################################################################")
print("DataFrame Rooms")
display(df_rooms)
print("################################################################################################################")
print("DataFrame Courses")
display(df_courses)
print("################################################################################################################")
print("DataFrame Configs")
display(df_configs)
print("################################################################################################################")
print("DataFrame Subparts")
display(df_subparts)
print("################################################################################################################")
print("DataFrame Classes")
display(df_classes)
print("################################################################################################################")
print("DataFrame Distributions")
display(df_distributions)
print("################################################################################################################")
print("DataFrame Students")
display(df_students)
print("################################################################################################################")
print("DataFrame RelationClassRoom")
display(df_relation_class_room)
print("################################################################################################################")
print("DataFrame RelationClassTime")
display(df_relation_class_time)
print("################################################################################################################")

################################################################################################################
DataFrame Problem


,Name,NrDays,SlotsPerDay,NrWeeks
0,agh-ggis-spr17,7,288,16


################################################################################################################
DataFrame Optimization


,Time,Room,Distribution,Student
0,4,1,15,5


################################################################################################################
DataFrame Rooms


...

################################################################################################################
DataFrame RelationClassRoom


,ClassId,RoomId,RoomPenalty
0,1,43,0
1,2,43,0
2,3,43,0
3,4,43,0
4,5,43,0
...,...,...,...
12372,1833,32,0
12373,1835,24,0
12374,1838,24,0
12375,1839,24,0


################################################################################################################
DataFrame RelationClassTime


,ClassId,TimeDays,TimeStart,TimeLength,TimeWeeks,TimePenalty
0,1,1000000,96,18,0101010010010101,0
1,1,0100000,96,18,0101010010010101,0
2,1,0010000,96,18,0101010010010101,0
3,1,0000100,96,18,0101010010101010,0
4,1,1000000,106,18,0101010010010101,0
...,...,...,...,...,...,...
46662,1836,0100000,176,18,0101010010010101,0
46663,1837,0100000,156,18,1010101000101010,0
46664,1838,0100000,176,18,0101010010010101,0
46665,1839,0100000,156,18,1010101000101010,0


################################################################################################################


Lectura xml solucion

In [2]:
import pandas as pd
import xml.etree.ElementTree as etree

tree = etree.parse("solution-tg-fal17.xml")
root = tree.getroot()

columns_sol = ["ClassId", "Days", "Start", "Weeks", "Room"]

df_sol = pd.DataFrame(columns = columns_sol)

for clase in root:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-2-2edd5ad0955c>, line 12)

Analisis de datos

In [6]:
## Duracion en slots de clases:

df_relation_class_time["TimeLength"] = df_relation_class_time["TimeLength"].astype(int)
print("Promedio:")
display(df_relation_class_time["TimeLength"].mean())
print("Min:")
display(df_relation_class_time["TimeLength"].min())
print("Max:")
display(df_relation_class_time["TimeLength"].max())
print("Tiempo de inicio")
display(df_relation_class_time.groupby("TimeStart").count())
print("Count")
display(df_relation_class_time["TimeLength"].value_counts())
print("Mediana:")
display(df_relation_class_time["TimeLength"].median())
print("Cuartiles:")
display(df_relation_class_time["TimeLength"].quantile([0, 0.25, 0.5, 0.75, 1]))
print("Var:")
display(df_relation_class_time["TimeLength"].var())
print("Std:")
display(df_relation_class_time["TimeLength"].std())

Promedio:


22.044592538624723

Min:


9

Max:


...

0.00      9.0
0.25     18.0
0.50     18.0
0.75     27.0
1.00    144.0
Name: TimeLength, dtype: float64

Var:


30.098277180969482

Std:


5.486189677815513

In [2]:
## Disponibilidad de salas:

for index, row in df_rooms.iterrows():
    print(row["RoomId"], row["UnavailableDaysList"])
    #if row["UnavailableDaysList"] is None:
    #    print(row["RoomId"], row["UnavailableDaysList"])

1 ['1000000', '0100000', '0100000', '0010000', '0001000', '0001000', '0000100', '0000010', '0000001']
2 ['1000000', '0100000', '0010000', '0001000', '0000100', '0000010', '0000010', '0000001']
3 ['1000000', '0100000', '0010000', '0001000', '0000100', '0000010', '0000010', '0000001', '0000001']
4 ['0000100', '0000010', '0000010', '0000001', '0000001', '1000000', '1000000', '0100000', '0001000', '0000100', '0000100', '0000100', '0000100', '0000100', '0000010', '0000010', '0000010', '0000010', '0000001', '0000001', '0000001', '0000001', '0000001']
5 ['0000100', '0000010', '0000010', '0000001', '0000001', '0100000', '0010000', '0000100', '0000100', '0000100', '0000100', '0000010', '0000010', '0000010', '0000010', '0000001']
6 ['1000000', '0100000', '0010000', '0001000', '0000100', '0000010', '0000001']
7 ['1000000', '0100000', '0010000', '0010000', '0001000', '0000100', '0000010', '0000001']
8 ['1000000', '1000000', '1000000', '0100000', '0010000', '0001000', '0000100', '0000010', '0000001

In [243]:
## Variabilidad de clases:

#for index, row in df_relation_class_time.iterrows():
#    print(row["ClassId"], row["TimeDays"])
display(df_relation_class_time["ClassId"].value_counts())

400     520
629     160
628     160
593     160
310     128
       ... 
321       1
463       1
1800      1
1804      1
1715      1
Name: ClassId, Length: 1852, dtype: int64

In [ ]:
#################### Nueva Parte versión Pau #######################
print("Slots de tiempo | Cantidad de cursos que duran esos slots")
dic={}
#list=[]
for index, row in df_relation_class_time.iterrows():
    #print(row["ClassId"], row["TimeLength"])
    if row["TimeLength"] not in dic.keys():
        dic[row["TimeLength"]] = [0,[]]
    if row["ClassId"] not in dic[row["TimeLength"]][1]:
        #print(row["TimeLength"],row["ClassId"], dic[row["TimeLength"]][1])
        dic[row["TimeLength"]][0]+=1
        dic[row["TimeLength"]][1].append(row["ClassId"])
c= 0
for i in dic:
    print(i, dic[i][0])
    c += dic[i][0]
print("Total",c)

In [153]:
#Agrupar capacidad de salas


print("Capacidad | Cantidad de Salas")
display(df_rooms.groupby("Capacity").count()["RoomId"])

dic_salas = {}
for index, row in df_rooms.iterrows():
    x = int(row["Capacity"])
    if x < 30:
        if "XS" not in dic_salas.keys():
            dic_salas["XS"] = [0,[]]
        dic_salas["XS"][1].append(row["RoomId"])
        dic_salas["XS"][0]+=1
    elif x == 30:
        if "S" not in dic_salas.keys():
            dic_salas["S"] = [0,[]]
        dic_salas["S"][1].append(row["RoomId"])
        dic_salas["S"][0]+=1
    elif x <= 90:
        if "M" not in dic_salas.keys():
            dic_salas["M"] = [0,[]]
        dic_salas["M"][1].append(row["RoomId"])
        dic_salas["M"][0]+=1
    else:
        if "L" not in dic_salas.keys():
            dic_salas["L"] = [0,[]]
        dic_salas["L"][1].append(row["RoomId"])
        dic_salas["L"][0]+=1
        
for i in sorted(dic_salas):
    print(i, dic_salas[i])

Capacidad | Cantidad de Salas


Capacity
120     1
140     1
15      9
180     2
240     1
25      1
28      2
29      1
30     13
35      3
42      3
45      1
70      1
90      3
94      1
96      1
Name: RoomId, dtype: int64

L [7, ['1', '3', '4', '5', '23', '24', '29']]
M [11, ['10', '11', '12', '14', '15', '16', '35', '40', '41', '42', '44']]
S [13, ['2', '9', '18', '19', '22', '28', '30', '31', '32', '34', '36', '37', '38']]
XS [13, ['6', '7', '8', '13', '17', '20', '21', '25', '26', '27', '33', '39', '43']]


In [4]:
#Agrupar capacidad de clases

print("Límite de las clases | Cantidad de clases con el límite")
display(df_classes.groupby("ClassLimit").count()["ClassId"])


Límite de las clases | Cantidad de clases con el límite


ClassLimit
10      29
100     12
11      12
12      46
120     18
128     59
129      3
13     100
130      8
14      10
15     223
18      18
180     20
20      66
21      64
22      93
24      70
25      41
26      75
28      56
30     360
32      20
36      79
38       4
40      19
42      54
43      18
44      18
48      39
50      16
52      45
60      76
64      60
75      18
90       3
Name: ClassId, dtype: int64

In [262]:
#Clases en horario similar -3 slots- de margen
#Tiempo inicio | Frecuencia de clases en ese horario | Id de clase
dicct = {}
df_relation_class_time["TimeStart"] = df_relation_class_time["TimeStart"].astype(int)
for index, row in df_relation_class_time.sort_values('TimeStart').iterrows():
    #print(row["ClassId"], row["TimeStart"])
    x1 = row["TimeStart"]-1
    x2 = row["TimeStart"]-2
    x3 = row["TimeStart"]-3
    if x1 in dicct.keys():
        if row["ClassId"] not in dicct[x1][1]:
            dicct[x1][1].append(row["ClassId"])
            dicct[x1][0] += 1
    elif x2 in dicct.keys():
        if row["ClassId"] not in dicct[x2][1]:
            dicct[x2][1].append(row["ClassId"])
            dicct[x2][0] += 1
    elif x3 in dicct.keys():
        if row["ClassId"] not in dicct[x3][1]:
            dicct[x3][1].append(row["ClassId"])
            dicct[x3][0] += 1
    elif row["TimeStart"] not in dicct.keys():
        dicct[row["TimeStart"]] = [0,[]]
        dicct[row["TimeStart"]][1].append(row["ClassId"])
        dicct[row["TimeStart"]][0] += 1
    elif row["TimeStart"] in dicct.keys() and row["ClassId"] not in dicct[row["TimeStart"]][1]:
        dicct[row["TimeStart"]][1].append(row["ClassId"])
        dicct[row["TimeStart"]][0] += 1

print("Tiempo inicio | Frecuencia de clases en ese horario | Lista de clase")
for i in dicct:
    print(i, dicct[i][0])

Tiempo inicio | Frecuencia de clases en ese horario | Lista de clase
93 1446
99 1
105 1349
111 2
116 1483
120 1
126 1440
132 7
136 1471
141 6
146 1437
150 9
156 1472
162 1
166 1362
171 5
176 1369
180 1
186 1327
190 2
195 1326
201 23
206 1242
210 2
216 1239
222 1
226 1203
231 1
236 1213
240 1
246 610
256 3


In [6]:
#Cantidad de clases en una subparte
c = 0
dics = {}
dicc_s = {}
for index, row in df_subparts.iterrows():
    dicc_s[row["SubpartId"]] = row["ClassIdList"]
    
for index, col in df_configs.iterrows():
    for i in col["SubpartIdList"]:
        if len(dicc_s[i]) not in dics.keys():
            dics[len(dicc_s[i])] = [1, [], [int(col["ConfigId"])]]
            for z in dicc_s[i]:
                dics[len(dicc_s[i])][1].append(z)
                dics[len(dicc_s[i])][0]+=1
        else:
            dics[len(dicc_s[i])][2].append(int(col["ConfigId"]))
            for z in dicc_s[i]:
                dics[len(dicc_s[i])][1].append(z)
                dics[len(dicc_s[i])][0]+=1


print("Cantidad de clases en una subparte | Cantidad de veces que se repite en las subpartes | Lista de clases | Config a la que pertenece ")
for i in sorted(dics):
    print(i, dics[i][0])

Cantidad de clases en una subparte | Cantidad de veces que se repite en las subpartes | Lista de clases | Config a la que pertenece 
1 882
2 451
3 121
4 201
5 56
6 37
8 25
10 51
12 37


In [260]:
#Cantidad de subpartes de una configuración

c = 0
dics = {}
dicc_s = {}
for index, row in df_subparts.iterrows():
    dicc_s[row["SubpartId"]] = row["ClassIdList"]
    
for index, col in df_configs.iterrows():
        #print(col["ConfigId"],col["SubpartIdList"])
        if len(col["SubpartIdList"]) not in dics.keys():
            dics[len(col["SubpartIdList"])] = [0, [], [int(col["ConfigId"])], 1]
            for z in col["SubpartIdList"]:
                dics[len(col["SubpartIdList"])][0]+=1
                for w in dicc_s[z]: 
                    dics[len(col["SubpartIdList"])][1].append(w)
                    dics[len(col["SubpartIdList"])][3]+=1
        else:
            dics[len(col["SubpartIdList"])][2].append(int(col["ConfigId"]))
            for z in col["SubpartIdList"]:
                dics[len(col["SubpartIdList"])][0]+=1
                for w in dicc_s[z]: 
                    dics[len(col["SubpartIdList"])][1].append(w)
                    dics[len(col["SubpartIdList"])][3]+=1


print("Cantidad de subpartes en una config | Cantidad de veces que se repite | Lista de clases | Config a la que pertenece ")
c=0
for i in sorted(dics):
        print(i, dics[i][0])
        c+=dics[i][0]
#print(dics)



Cantidad de subpartes en una config | Cantidad de veces que se repite | Lista de clases | Config a la que pertenece 
1 32
2 258
3 45
4 16
5 45
6 96
7 49
8 24
9 207
10 70
11 33
12 144
13 13
14 14
15 105
18 18
26 26
29 29


In [246]:
import math
#Agrupar capacidad de cursos

#print("Límite de las clases | Cantidad de clases con el límite")
#display(df_classes.groupby("ClassLimit").count()["ClassId"])

dicc_cursos = {}
#display(df_courses)
for index, row in df_courses.iterrows():
    dicc_cursos[row["CourseId"]] = row["ConfigIdList"]
#print(dicc_cursos)      
dicc_configs = {}
#display(df_configs)
for index, row in df_configs.iterrows():
    dicc_configs[row["ConfigId"]] = row["SubpartIdList"]
#print(dicc_configs)
dicc_subparts = {}
#display(df_subparts)
for index, row in df_subparts.iterrows():
    dicc_subparts[row["SubpartId"]] = row["ClassIdList"]
#print(dicc_subparts)
dicc_classes = {}
#display(df_classes)
for index, row in df_classes.iterrows():
    dicc_classes[row["ClassId"]] = row["ClassLimit"]
#print(dicc_classes)

dicc_lim = {}
dicc_curso_clase = {}
for p,i in dicc_cursos.items():
    for j in i:
        if j in dicc_configs.keys():
            #print(dicc_configs[j])
            for k in dicc_configs[j]:
                if k in dicc_subparts.keys():
                    #print(dicc_subparts[k])
                    for l in dicc_subparts[k]:
                        #print(l)
                        if l not in dicc_curso_clase:
                            dicc_curso_clase[l] = []
                        dicc_curso_clase[l].append(p)
                        if l in dicc_classes.keys():
                            n = int(dicc_classes[l])
                            if p not in dicc_lim.keys():
                                dicc_lim[p] = [n]
                            else:
                                dicc_lim[p].append(n)
                        
#print(dicc_lim)
#print("Id de curso | Limite de alumnos")
for i in dicc_lim.keys():
    suma = 0
    contador = 0
    for j in dicc_lim[i]:
        suma += j
        contador += 1
    promedio = suma/contador
    dicc_lim[i] = int(math.ceil(promedio/10.0))*10
   # print(i, dicc_lim[i])
#print(dicc_curso_clase

In [264]:
cluster = {}
#dics subparte
#dicct horario
for i in sorted(dics):
    for j in dics[i][1]:
        #print(dics[i][1
        for x in dicct:
            for w in dicct[x][1]:
                if j == w:
                    #tupla2 = tuple(set(dics[i][2]))
                    key = (i,x)
                    if key not in cluster.keys():
                        cluster[key] = []
                    #print(dics[i][2])
                    cluster[key].append(dicc_curso_clase[j][0])
                
#print("(Cursos, cantidad de clases de subparte, horario): Clases que se adaptan")
#print(cluster)



In [208]:
#Patrones de salas
diccr = {}
dic2 = {}
for index, row in df_relation_class_room.iterrows():
    #print(row["ClassId"], row["RoomId"])
    if row["ClassId"] not in diccr.keys():
        diccr[row["ClassId"]] = []
    diccr[row["ClassId"]].append(row["RoomId"])
#print(diccr)

for i in diccr:
    #print(i, diccr[i])
    tupla = tuple(diccr[i])
    if tupla not in dic2.keys():
        dic2[tupla] = []
    dic2[tupla].append(i)
#print(dic2)

c = 0

#print("Salas | Preferencias de clases por la sala")
#for i in dic2:
#    print(i, dic2[i])
#    c+=1
#print(c)

In [162]:
## Cuantos cursos hay para cada capacidad redondeada

print("Capacidad de curso | Conteo de cursos")
dicc_cont = {}
for i in dicc_lim.values():
        if i not in dicc_cont.keys():
            dicc_cont[i] = 1
        else:
            dicc_cont[i] += 1
for i in dicc_cont.keys():
    print(i, dicc_cont[i])

Capacidad de curso | Conteo de cursos
20 21
60 20
50 35
30 128
130 6
120 3
180 2
40 46
10 1
80 8
70 2


In [156]:
## Cuantos cursos hay para cada capacidad redondeada

print("Capacidad de curso | Conteo de cursos | Cursos")
dicc_cont2 = {}
for i in dicc_lim:
    if dicc_lim[i] not in dicc_cont2.keys():
        dicc_cont2[dicc_lim[i]] = [int(i)]
    else:
        #dicc_cont2[dicc_lim[i]][0] += 1
        dicc_cont2[dicc_lim[i]].append(int(i))
#for i in dicc_cont2.keys():
#    print(i, dicc_cont2[i])

Capacidad de curso | Conteo de cursos | Cursos


In [255]:
cluster2 = {}
c = 0
#for i in cluster:
#    cluster2[(i[1],i[2])] = list(i[0])
#for i in cluster2:
#    print(i, cluster2[i],"\n")
#print(cluster2)
#cluster2 = cluster
for i in cluster:
    cluster2[i] = set(cluster[i])
print(cluster2)

{(1, 93): {'57', '59', '72', '29', '91', '1', '195', '35', '134', '45', '41', '73', '87', '241', '240', '166', '19', '153'}, (1, 105): {'57', '59', '29', '1', '195', '35', '45', '41', '73', '87', '166', '72', '153'}, (1, 116): {'57', '59', '72', '29', '1', '195', '35', '134', '45', '41', '73', '87', '96', '80', '166', '3', '153'}, (1, 126): {'57', '59', '29', '1', '195', '35', '45', '41', '73', '87', '80', '166', '72', '153'}, (1, 136): {'57', '59', '29', '1', '195', '35', '220', '45', '41', '73', '87', '80', '166', '72', '153'}, (1, 146): {'57', '59', '29', '1', '195', '35', '45', '41', '73', '87', '80', '166', '72', '153'}, (1, 156): {'57', '59', '267', '269', '29', '266', '1', '271', '195', '35', '45', '41', '73', '87', '92', '17', '80', '153'}, (1, 166): {'59', '29', '1', '195', '35', '45', '41', '73', '87', '80', '153'}, (1, 176): {'57', '59', '29', '1', '195', '35', '272', '45', '41', '73', '87', '268', '270', '153'}, (1, 186): {'57', '59', '29', '1', '195', '45', '41', '153'}, (

In [265]:
print("Subparte | horario | límite clase")
cluster_cursos = {}
for i in cluster2:
    #print(i,cluster2[i])
    for j in cluster2[i]:
        c = 0
        for k in dicc_cont2:
            for z in dicc_cont2[k]:
            #print(k,dicc_cont2[k])
                if int(j) == int(z):
                    c+=1
                    key = i[0],i[1],k
                    if key not in cluster_cursos:
                        cluster_cursos[key] = []
                    cluster_cursos[key].append(z)
                    #print(i[0],i[1],j)
                    #cluster_cursos[(i[0],i[1],j)] = dicc_cont2[k]

#print(cluster_cursos)



Subparte | horario | límite clase


In [259]:
print("(Subparte | horario | límite alumnos curso): [Cursos]")
lista = []
c=0
for i in cluster_cursos:
    #print(cluster_cursos[i])
    for j in cluster_cursos[i]:
        if j not in lista:
            lista.append(j)
            c+=1
for i in cluster_cursos:
    print(i, cluster_cursos[i],"\n")
 #   print(i, cluster_cursos[i])
print(len(cluster_cursos))
print(c)
print(sorted(lista))
#cluster_cursos2 = {}
#for i in cluster_cursos:
 #   if i in cluster_cursos2.keys()
  #  print(i, cluster_cursos[i],"\n")

(Subparte | horario | límite alumnos curso): [Cursos]
(1, 93, 30) [57, 59, 29, 35, 134, 45, 41, 240, 166, 153] 

(1, 93, 40) [72, 73, 87] 

(1, 93, 60) [91] 

(1, 93, 20) [1] 

(1, 93, 50) [195, 241] 

(1, 93, 180) [19] 

(1, 105, 30) [57, 59, 29, 35, 45, 41, 166, 153] 

(1, 105, 20) [1] 

(1, 105, 50) [195] 

(1, 105, 40) [73, 87, 72] 

(1, 116, 30) [57, 59, 29, 35, 134, 45, 41, 80, 166, 153] 

(1, 116, 40) [72, 73, 87] 

(1, 116, 20) [1, 3] 

(1, 116, 50) [195] 

(1, 116, 60) [96] 

(1, 126, 30) [57, 59, 29, 35, 45, 41, 80, 166, 153] 

(1, 126, 20) [1] 

(1, 126, 50) [195] 

(1, 126, 40) [73, 87, 72] 

(1, 136, 30) [57, 59, 29, 35, 45, 41, 80, 166, 153] 

(1, 136, 20) [1] 

(1, 136, 50) [195, 220] 

(1, 136, 40) [73, 87, 72] 

(1, 146, 30) [57, 59, 29, 35, 45, 41, 80, 166, 153] 

(1, 146, 20) [1] 

(1, 146, 50) [195] 

(1, 146, 40) [73, 87, 72] 

(1, 156, 30) [57, 59, 267, 269, 29, 266, 35, 45, 41, 80, 153] 

(1, 156, 20) [1] 

(1, 156, 60) [271] 

(1, 156, 50) [195] 

(1, 156, 40) [

(14, 116, 30) [206] 

(14, 126, 30) [206] 

(14, 136, 30) [206] 

(14, 146, 30) [206] 

(14, 156, 30) [206] 

(14, 166, 30) [206] 

(14, 176, 30) [206] 

(14, 186, 30) [206] 

(14, 195, 30) [206] 

(14, 206, 30) [206] 

(14, 216, 30) [206] 

(14, 226, 30) [206] 

(14, 236, 30) [206] 

(15, 93, 30) [174, 182, 222, 250] 

(15, 93, 20) [217, 196] 

(15, 93, 50) [215] 

(15, 105, 30) [174, 182, 222, 250] 

(15, 105, 20) [217, 196] 

(15, 105, 50) [215] 

(15, 116, 30) [174, 182, 222, 250] 

(15, 116, 20) [217, 196] 

(15, 116, 50) [215] 

(15, 126, 30) [174, 182, 222, 250] 

(15, 126, 20) [217, 196] 

(15, 126, 50) [215] 

(15, 136, 30) [174, 182, 222, 250] 

(15, 136, 20) [217, 196] 

(15, 136, 50) [215] 

(15, 146, 30) [174, 182, 222, 250] 

(15, 146, 20) [217, 196] 

(15, 146, 50) [215] 

(15, 156, 30) [174, 182, 222, 250] 

(15, 156, 20) [217, 196] 

(15, 156, 50) [215] 

(15, 166, 30) [174, 182, 222, 250] 

(15, 166, 20) [217, 196] 

(15, 166, 50) [215] 

(15, 176, 30) [174, 182, 222,

In [136]:
dic_cluster = {}
for i, j in enumerate(cluster_cursos):
    dic_cluster[i] = cluster_cursos[j]

In [131]:
from gurobipy import *
from random   import *
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [ ]:
timetabling = gp.Model("Asignacion de cursos a horarios y salas")

#indices por variable
# x: (cluster,config,bloque)
# w: (cluster,curso)
# c: (config,bloque,patron)


#diccionario cluster {id cluster: cursos dentro}
# config {id curso: }
# conjuntos/subconjuntos
dic_cluster = 
config = 
bloque = 
curso = 
patron = 
Gc_patron = 
Hd_bloque = 
Z =

# parámetros
Q_f = 
M = 100000000000
QN_zp = #hacer como tupleList yo creo
QS_z =
L_c =
h_c = 

## variables

x = {}
for c in dic_cluster.keys():
    for f in range(len(config)):
        for b in range (len(bloque)):
            x[c,f,b] = timetabling.addVar(vtype=grb.GRB.BINARY,name='x_'+str(c)+''+str(f)+''+str(b))
w = {}
for c in range(len(cluster)):
    for k in range(len(curso)):
        w[c,k] = timetabling.addVar(vtype=grb.GRB.BINARY,name='w_'+str(c)+'_'+str(k))
y = {}
for f in range(len(config)):
    for b in range(len(bloque)):
        for p in range(len(patron)):
            y[f,b,p] = timetabling.addVar(vtype=grb.GRB.BINARY,name='y_'+str(f)+''+str(b)+''+str(p))


#Función Objetivo
N= 1 # ???
timetabling.setObjective(N,GRB.MINIMIZE)

#Restricciones

timetabling.addConstr((gp.quicksum(y[f,b,p] for p in range(len(Gc_patron))) <= 1), name = "R1")
timetabling.addConstr((gp.quicksum(x[c,f,b] for b in range(len(Hd_bloque))) == Qf), name = "R2")
timetabling.addConstr((x[c,f,b] <= M*y[f,b,p]), name = "R3")
timetabling.addConstr((y[f,b,p]*(gp.quicksum(QN_zp[z,p]*QS_z[z] for z in range(len(Z)))) <= x*L_c), name = "R4") 
timetabling.addConstr((h_c + gp.quicksum(QN_zp[z,p]*QS_z[z] for z in range(len(Z)))>= x*L_c), name = "R5")
timetabling.addConstr(gp.quicksum(x[c,f,b] for c in range(len(cluster))) == gp.quicksum(y[f,b,p] for p in range(len(Gc_patron))), name = "R6")
